# 一、神经网络基本原理
## 1.1 什么是神经网络？
神经网络启发于生物学的神经元连接，通过大量的模型参数学习数据中的有效特征来完成对应的任务。核心过程包括前向传播和后向传播，前向传播通过设计各种模型结构来更有效的特征提取，后向传播根据损失函数完成能够降低任务损失的参数更新。
## 1.2 反向传播算法（Backpropagation）
反向传播是基于链式法则的参数梯度计算方法。前向传播阶段会构建计算图，记录各层的操作与中间结果；输出通过损失函数聚合为一个标量 loss。随后，反向传播沿计算图自后向前递推loss对每层参数的梯度。最终，这些梯度通过梯度下降等优化算法用于更新模型参数。
- 链式法则是用于求复合函数导数的公式，表示复合函数的导数等于各个内部函数导数的乘积。在深度学习中，它是反向传播算法的数学基础，用来将损失函数对输出的梯度，通过层与层之间的依赖关系，一层层地传回去，最终得到每个参数的梯度，从而进行梯度下降优化。
- 为什么是最后的loss是标量？
    - 标量才可以有唯一方向降低损失：
        
        假设神经网络最终输出是：
        - $\hat{y} = f(x; \theta)$
        - 损失函数：$L = \text{Loss}(\hat{y}, y)$

        需要计算：

        $$
        \frac{\partial L}{\partial \theta}
        $$

        这只能在 $L$ 是标量的情况下才能得到明确的导数，保证有唯一方向去更新参数。如果 $L$ 是向量，比如 shape = (batch\_size,)，将无法直接用链式法则去求导。

- 我理解的误区：每个样本的loss组成的向量并不是进行反向传播的损失L，我们能够直接根据损失函数求每个样本的导数，但其导数也是对于聚合后的、反向传播的损失L
    
    假如定义的 loss 是：

    $$
    L = \sum_{i=1}^n \frac{1}{2}(y_i - \hat{y}_i)^2
    $$

    我们想要求：

    $$
    \frac{\partial L}{\partial \hat{y}_i}
    $$


    - 方法一：人类理解的

        对每一项求导：

        $$
        \frac{\partial L}{\partial \hat{y}_i} = \hat{y}_i - y_i
        $$

        完全正确，没问题！


    - 方法二：自动求导框架的实际“链式传播路径”

        框架是这样理解你的写法的：

        1. **先计算每个样本的损失：**

        $$
        L_i = \frac{1}{2}(y_i - \hat{y}_i)^2
        $$

        2. **再通过聚合：**

        * 使用 `sum`：

            $$
            L = \sum_i L_i
            $$

            对 $L_i$ 的导数是 $\frac{\partial L}{\partial L_i} = 1$

        3. **再通过链式法则：**

        $$
        \frac{\partial L}{\partial \hat{y}_i} = \frac{\partial L}{\partial L_i} \cdot \frac{\partial L_i}{\partial \hat{y}_i} = 1 \cdot (\hat{y}_i - y_i)
        $$

        所以结果依然是：
        >
        > $$
        > \boxed{\frac{\partial L}{\partial \hat{y}_i} = \hat{y}_i - y_i}
        > $$
        >
        > 和你直觉上的结果完全一样。


## 1.3 梯度下降再认识
### 1️⃣ 梯度的定义

对标量 loss $L$ 关于参数张量 $W$ 求导：

$$
\nabla_W L = \frac{\partial L}{\partial W}
$$

* 结果是与参数同形状的张量，每个元素对应一个参数的偏导数
* 整个梯度张量在高维参数空间中构成了 loss 上升最快的方向，负梯度方向就是下降最快的方向
* 每个元素 $\frac{\partial L}{\partial W_{ij}}$ 是整个梯度向量的一个分量，loss这个分量下增加的瞬时变化率（即如果只改变该参数一个单位，loss 的变化量）

**一句话总结：** 在深度学习中，loss 是参数张量的函数，其梯度是一个向量，表示当前参数点处 loss 上升最快的方向。每个参数的偏导数（即梯度分量）表示该参数对 loss 的局部影响程度，整体梯度方向指引我们如何更新参数以最快地减小 loss。

**形象比喻：**

想象三维空间的地形（loss landscape）：

* 你站在某一点 $W$ 

* 沿 x、y、z 方向，你能测到坡度（梯度分量）

* 把三个方向的坡度合成一个箭头（梯度向量）

* 顺着负梯度箭头走，就能最快下山（loss下降）


---

### 2️⃣ 梯度下降更新

梯度下降是利用梯度信息更新参数，使 loss 下降最快的方法。具体步骤如下：

1. **计算梯度**
   对当前参数 $W$ 计算 loss 的梯度：

   $$
   \nabla_W L = \frac{\partial L}{\partial W}
   $$

2. **沿负梯度方向更新参数**
   使用学习率 $\eta$ 控制步长，更新公式为：

   $$
   W \gets W - \eta \nabla_W L
   $$

   * 这里 $-\nabla_W L$ 是 loss 下降最快的方向。
   * 学习率 $\eta$ 决定每一步走多远，过大可能发散，过小收敛慢。

3. **重复迭代**
   在训练过程中，多次计算梯度并更新参数，逐步减小 loss，直到满足停止条件（如 loss 足够小或达到最大迭代次数）。

---

**形象比喻：**

* 想象你站在三维地形的某一点：

  * 梯度告诉你“地面坡度有多陡、哪个方向坡最大”。
  * 负梯度方向就是下山最快的路。
  * 每次沿这个方向走一步，就相当于更新参数 $W$ 一次。
* 反复多次，就像沿山坡下滑，最终到达低谷（loss 最小）。




## 1.4 激活函数

1. 什么是激活函数？

在神经网络中，**激活函数**是连接在每一层线性变换（如加权求和）之后的**非线性函数**，它的作用包括：

- 引入非线性，使网络可以拟合更复杂的函数关系
- 控制输出的尺度范围，防止数值爆炸
- 有些激活函数还具有稀疏性或抑制特征，有助于正则化。比如relu有稀疏性，把小于0给屏蔽掉；sigmoid有异质性

---

2. 饱和区和敏感区（线性区）

- **饱和区（Saturation Region）：**
  - 输入极大或极小时，**输出趋于常数**，此时梯度（导数）趋近于 0，容易导致 **梯度消失问题**
  - 常见于 Sigmoid、Tanh 等激活函数的两端
  - 表示网络对输入**不敏感**

- **敏感区 / 线性区（Linear/Sensitive Region）：**
  - 激活函数梯度较大、变化快的区域
  - 神经元对输入变化较为敏感，易于学习
  - 例如：Sigmoid 在 \(x=0\) 附近；Tanh 在中心附近

---

3. 常见损失函数


| 函数名 | 表达式 | 输出范围 | 饱和区区间 | 敏感区 | 常用损失函数 | 用途说明 | 优缺点总结 |
|--------|--------|-----------|--------------|------------|----------------|------------|------------|
| **Sigmoid** | $f(x) = \frac{1}{1 + e^{-x}}$ | (0, 1) | $x \ll 0$ 或 $x \gg 0$ | $x \approx 0$ | Binary Cross Entropy | 二分类输出层 | ✅ 输出概率，易理解；❌ 梯度消失，两端饱和，不零中心 |
| **Tanh** | $f(x) = \frac{e^x - e^{-x}}{e^x + e^{-x}}$ | (−1, 1) | $x \ll 0$ 或 $x \gg 0$ | $x \approx 0$ | MSE 或 BCE | RNN常用，零中心 | ✅ 零中心输出；❌ 梯度消失，饱和区存在 |
| **ReLU** | $f(x) = \max(0, x)$ | [0, ∞) | 无（负区间恒为0） | $x > 0$ | MSE、CE | 默认首选，卷积/全连接 | ✅ 简单高效，梯度不消失；❌ 负区间神经元可能死亡 |
| **Leaky ReLU** | $f(x) = \max(\alpha x, x), \alpha \approx 0.01$ | (−∞, ∞) | 无 | 全区间 | MSE、CE | 缓解死神经元 | ✅ 保持梯度活性，训练鲁棒；❌ 输出非零中心，仍可能过大 |
| **ELU** | $f(x) = \begin{cases} x,& x>0 \\ \alpha (e^x-1),& x\le0 \end{cases}$ | (−α, ∞) | $x \ll 0$ | $x \approx 0$ | MSE、CE | 零中心，训练更平滑 | ✅ 零中心，平滑；❌ 计算稍慢，负区指数计算 |
| **Softmax** | $f(x_i) = \frac{e^{x_i}}{\sum_j e^{x_j}}$ | (0,1), 和为1 | 指数项可能导致放大 | 全维度 | Cross Entropy | 多分类输出层 | ✅ 转化为概率分布；❌ 指数运算可能数值不稳定 |







## 1.5 权重初始化
1. 什么是权重初始化？
在深度神经网络中，每一层都有可学习的参数，在训练开始的时候需要赋予这些参数初始值，这就是权重初始化。不合理的权重初始化，会导致梯度消失、梯度爆炸和神经网络学习不到有意义的特征（如常数随机初始化），所以在随机初始化的时候应该保证：
- 激活值不过大，也不过小
- 正向传播和反向传播信号不会放大或者衰减
- 神经元有“差异性”，能学习不同特征

---

2. 常见的初始化方法

| 初始化方法     | 初始化公式（正态分布）                                  | 初始化公式（均匀分布）                                    | 适用激活函数        | 特点说明                                 |
|----------------|--------------------------------------------------------|-----------------------------------------------------------|---------------------|------------------------------------------|
| 随机初始化     | $W \sim \mathcal{N}(0, 1)$                              | $W \sim U(-1, 1)$                                          | 通用（但不推荐）    | 简单易用，但容易梯度爆炸或消失           |
| Xavier 初始化  | $W \sim \mathcal{N}(0, \frac{2}{n_{in} + n_{out}})$    | $W \sim U\left(-\sqrt{\frac{6}{n_{in}+n_{out}}}, \sqrt{\frac{6}{n_{in}+n_{out}}}\right)$ | Sigmoid, Tanh       | 保持前向/反向传播方差一致，适合饱和激活  |
| He 初始化       | $W \sim \mathcal{N}(0, \frac{2}{n_{in}})$              | $W \sim U\left(-\sqrt{\frac{6}{n_{in}}}, \sqrt{\frac{6}{n_{in}}}\right)$ | ReLU, Leaky ReLU    | 专为ReLU设计，增强激活有效性，防止死神经元 |
| 常数初始化     | $W = c$                                                | -                                                         | ❌ 不推荐           | 所有神经元行为一致，无法学习差异         |
| 全零初始化     | $W = 0$                                                | -                                                         | ❌ 严重不推荐       | 所有神经元完全一致，训练完全失效         |
> 说明：
>
> * $n_{\text{in}}$：当前层输入神经元个数
> * $n_{\text{out}}$：当前层输出神经元个数
> * 你可以根据需要选用正态或均匀分布，两者都常见

---

3. 深入理解 Xavier 和 He 初始化

Xavier 和 He 初始化的核心目标是：
- 让不同层的激活值和梯度保持方差稳定，从而避免梯度爆炸或消失。
- 它们通过设定参数的初始分布方差，根据每层的输入/输出规模自动调整，来实现不同层激活值方差稳定的

神经网络每一层的输出是：
$$
y = W x + b
$$
**理想情况**是：输出的方差 **不增不减**，也就是说每一层输出值和输入值的方差接近相等。这对于 **前向传播** 和 **反向传播** 都非常关键。

- 🔷 Xavier 初始化（又叫 Glorot 初始化）

    * **设计思想：**

    > 同时考虑输入和输出节点数，保持激活值和梯度的方差在正反向传播中都不发生剧烈变化。

    * **适用函数：** Tanh、Sigmoid 这类激活函数可能饱和，梯度容易消失，因此初始化要温和。

    * **数学依据：**
    如果希望输出方差 ≈ 输入方差：

    $$
    Var(W) = \frac{2}{n_{\text{in}} + n_{\text{out}}}
    $$

    * **效果：**

    * 避免前向传播中激活值太大或太小
    * 避免反向传播中梯度爆炸或消失

- 🔶 He 初始化（又叫 Kaiming 初始化）

    * **设计思想：**

    > ReLU 会把一半的输出砍掉（负的变0），所以必须放大初始权重的方差来补偿这种“信息损失”。

    * **适用函数：** ReLU, Leaky ReLU

    * **数学依据：**
    为了保持激活值方差不变（考虑 ReLU 会减半激活）：

    $$
    Var(W) = \frac{2}{n_{\text{in}}}
    $$

    * **效果：**

    * 更适合 ReLU，输出不易全部变成 0
    * 提高训练初期的信号传播能力


---
4. 为什么初始化为常数或者0？

如果 所有权重相等（如常数或0）：
- 所有神经元计算出的值就完全一样
- 反向传播中得到的梯度也完全一样
- 导致神经元“退化”成一样的，没有学习能力
换句话说：初始化相同 → 神经元感受不到差异 → 学不到不同特征 → 网络失效。

---

5. 相邻两层的方差稳定能够通过Xavier和He初始化保证，但是均值又是怎么维持的呢？怎么能够保证前向和后向传播稳定的？

* **结论**：均值由权重均值和输入均值决定，方差由 Xavier/He 初始化保证，二者结合可以让每层输出既接近零中心又稳定。

假设一层神经元输出：

$$
z = \sum_{i=1}^{n} w_i x_i + b
$$

**条件假设**

* 权重 $w_i$ **均值为 0**
* 输入 $x_i$ 均值约为 0
* 权重与输入相互独立
* 偏置 $b$ 初始化为 0 或很小的值

**输出均值分析**

$$
\mathbb{E}[z] = \sum_{i=1}^{n} \mathbb{E}[w_i x_i] + \mathbb{E}[b] = \sum_i \mathbb{E}[w_i]\mathbb{E}[x_i] + 0 = 0
$$

✅ 因此，通过 **权重均值为 0** + **输入均值接近 0**，输出均值自然接近 0，即使层数增加，也不会产生较大偏差。

**输入均值为什么接近 0？**

* 输入层数据通常经过 **标准化（归一化 / 零均值化）**

  * 减去均值，除以标准差
  * 保证第一层输入均值接近 0，方差约为 1
* 对于深层网络，每层权重均值为 0 → 下一层输入均值仍保持在 0 附近
* **因此即使多层堆叠，均值不会偏离太大，保证了网络的稳定性**

**He初始化的些许不同**

He 初始化保证了 ReLU 激活的输出方差稳定，均值从上一层到输出层保持在 0 附近（由上面公式推导）。经过 ReLU 截断后，输出均值略偏正，但仍在 0 附近，加上方差稳定，所以激活值整体仍保持在合理范围，训练过程稳定。





## 1.6 标准化、归一化和正则化
### 1. 什么是标准化、归一化和正则化？
- 标准化：将原始的数据变成均值为0，方差为1的分布，该做法会改变数据的原始分布情况 
- 归一化：将数据压缩到固定的范围内，如 [0,1]、[−1,1] 等，该做法不会改变数据的原始分布，但异常点的影响很大。以上两者都能改变量纲带来的影响
- 正则化：通过在损失函数中增加约束项（如 L1/L2）限制模型参数大小，用于降低模型复杂度、防止过拟合，不会改变输入数据分布

### 2. 标准化和归一化的意义？（站在数据预处理角度）

| 作用     | 说明                             | 区别                     |
| ------ | ------------------------------ |----------------------
| 加快收敛速度 | 避免某些特征值太大或太小导致训练不稳定            |标准化是统一分布，数值都变小但是没有固定区间；归一化是固定到特定数值小区间 |
| 提高模型性能 | 标准化有助于梯度方向统一、学习更有效             | 类比在一个“椭圆形碗”里走向最低点（未标准化），优化会在长轴方向来回震荡；标准化后像在圆形碗中，能直接下滑到最低点 |
| 避免特征支配 | 特征尺度差异大会导致权重偏移                 | 两者都对数值尺度进行了减小，不会有明显偏向 |
| 适配某些模型 | SVM、KNN、PCA等对距离或方差敏感的模型需要标准化处理 |\
| 为参数初始化创造良好条件 | 标准化保证输入均值为0，方差为1，有利于 Xavier / He 初始化 的方差保持策略起效;归一化保证输出均值和方差很小来帮助初始化策略起作用 |\|

### 3. 常见标准化和归一化处理

| 方法            | 类型  | 数学公式                                         | 适用场景          | 特点              |
| ------------- | --- | -------------------------------------------- | ------------- | --------------- |
| Min-Max 归一化   | 归一化 | $x' = \frac{x - x_{min}}{x_{max} - x_{min}}$ | 图像像素、范围已知     | 缩放到固定区间，受极端值影响大 |               |                |
| Z-Score 标准化   | 标准化 | $x' = \frac{x - \mu}{\sigma}$                | 近似正态分布的数据     | 输出均值为 0，方差为 1   |
| Max Abs 缩放    | 归一化 | $x' = \frac{x}{\lvert x_{\max} \rvert}$ | 稀疏数据如文本TF-IDF | 保持稀疏性，仅缩放不移动均值 |
| Robust 标准化    | 标准化 | $x' = \frac{x - \text{median}}{\text{IQR}}$ median：数据中位数 IQR=Q3​−Q1：四分位距（中间 50% 数据范围） | 有异常值的数据       | 25% 到 75% 的“中间数据”映射到统一的数值范围内，从而抵抗异常值、保持特征稳定性     |
| log变换 | \ | $x' = \log(x + \epsilon)$ | 数据偏态分布，有异常值 | 对数缩放 → 压缩大数值、缓解偏态分布、减轻异常值影响、提高模型稳定性  |
| L2归一化   | 向量归一化 | $x' = \frac{x}{\|x\|_2}$              | 表征向量归一化 |保持方向不变，单位模长，多用于文本特征、嵌入向量等   |
| L1归一化   | 向量归一化 | $x' = \frac{x}{\|x\|_1}$              | |稀疏建模场景，保证每个样本特征绝对值和为1       |
> 说明：
> * L1归一化和L2归一化实际上是除以对应的范数
> * L1范数：$\|x\|_1 = \sum_i |x_i|$,取向量每个元素的**绝对值之和**
> * L2范数：\|x\|_2 = \sqrt{\sum_i x_i^2}，取向量的 **欧几里得模长**

### 4. 其它
| 方法     | 标准化对象    | 标准化维度    | 公式（核心）        | 主要作用        | 特点   |
| -------- | ------------ | ------------- | ---------------- | --------------- | ------------------ |
| **L1 正则**    | 权重参数 $w$ | 向量整体   | $\lambda \|w\|_1 = \lambda \sum_i \lvert w_i \rvert $  | 控制模型复杂度，稀疏性 | 可使部分权重为 0，有特征选择效果 |
| **L2 正则**  | 权重参数 $w$ | 向量整体    | $\lambda \|w\|_2^2 = \lambda \sum_i w_i^2$  | 控制模型复杂度，防止过拟合 | 平滑约束，保留所有特征        |
| **BatchNorm** | 激活 $x$   | 见说明 | $\hat{x} = \frac{x - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}}$ | 加速收敛，缓解内部协变量偏移           | 依赖 batch，训练/推理模式不同 |
| **LayerNorm**  | 激活 $x$   | 见说明    | $\hat{x} = \frac{x - \mu_L}{\sqrt{\sigma_L^2 + \epsilon}}$ | 稳定训练，常用于 RNN/Transformer | 不依赖 batch，适合序列模型   |
| **InstanceNorm** | 激活 $x$   |  见说明   | 和 BN 类似，但均值/方差是按单个图像     | 图像风格迁移等     | 保持样本独立性，不依赖 batch  |
> 说明：
> * L1 正则化：导数是 $\text{sign}(w)$（即 $+1$ 或 $-1$），在 $w=0$ 处不可导但有次梯度；需手动添加正则项 $\lambda |w|_1$ 到 loss 中，实现稀疏性。
> * L2 正则化：导数是 $2w$，对所有点可导；通常通过优化器中的 weight_decay 参数自动实现，用于限制权重大小，使训练更稳定。

假设图片为（B，C，W，H），文本为（B，L，D）
- BatchNorm
 - 对于图片来说是不同样本的同一通道的所有（W, H）进行标准化，维度是（N * W * H）
 - 对于文本来说是不同样本，L的相同位置词向量进行标准化, 维度是（N * D）
- LayerNorm
    - 对于图片来说是同一样本的所有通道的所有（W, H）进行标准化，维度是（C * W * H）
    - 对于文本来说是同一样本，每个词向量进行单独标准化，，维度是（1 * D）
- InstanceNorm
    - 对于图片来说每个样本每个通道 独立做标准化，维度是（1 * W * H）
    - 对于文本就是Layernorm，只不过在标准化过程中可学习的仿射参数不同
- LN和IN区别
    - LN和IN都是对[1, 1, D]个变量求均值和方差，得到一个[B, L, 1]形状的均值和方差tensor去做标准化。
    - 两者不同是在仿射变化一步，LN中的仿射变化参数beta和gamma的形状是[1, 1, D]，而IN中是[1, L, 1]





## 1.7 损失函数
### 1. 什么是损失函数及其作用？

损失函数（Loss Function）用于衡量模型输出与真实标签之间的差异，是训练神经网络时优化的目标函数。通过反向传播计算损失关于参数的梯度，引导模型学习更好的参数。

---
### 2. 常见损失函数

**回归任务**

| 损失函数        | 适用场景         | 特点说明                   |
| ----------- | ------------ | ---------------------- |
| MSE（均方误差）   | 连续数值预测       | 平滑、凸函数，对异常值敏感          |
| MAE（平均绝对误差） | 连续数值预测       | 对异常值鲁棒，但在零点不可导         |
| Huber Loss  | 连续数值预测，兼顾异常值 | 小误差用 MSE，大误差用 MAE，梯度稳定 |
| MAPE (平均绝对百分比误差) | 任何需要误差占比评价的回归任务 | 误差占真实值比例，适合衡量不同量级的数据,单位无关|

**MSE**

$$
\text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2
$$

**MAE**

$$
\text{MAE} = \frac{1}{n} \sum_{i=1}^{n} |y_i - \hat{y}_i|
$$

**Huber Loss**

$$
L_\delta(y, \hat{y}) = 
\begin{cases}
\frac{1}{2}(y - \hat{y})^2 & |y - \hat{y}| \le \delta \\
\delta(|y - \hat{y}| - \frac{1}{2}\delta) & \text{otherwise}
\end{cases}
$$

**MAPE**
$$
\text{MAPE} = \frac{100\%}{n} \sum_{i=1}^{n} \left| \frac{y_i - \hat{y}_i}{y_i} \right|
$$



**分类任务**

| 损失函数                             | 适用场景 | 特点说明                      |
| -------------------------------- | ---- | ------------------------- |
| Binary Cross Entropy (BCE)       | 二分类  | 搭配 Sigmoid 输出，错分惩罚大       |
| Categorical Cross Entropy (CCE)  | 多分类  | 搭配 Softmax 输出和 one-hot 标签 |
| Sparse Categorical Cross Entropy | 多分类  | 标签为整数索引，节省空间              |
>
> CCE是交叉熵损失函数 + softmax 输出，组合后导数更简洁，有 log-softmax 合并优化

**BCE**

$$
\text{BCE} = -[y \log(\hat{y}) + (1 - y) \log(1 - \hat{y})]
$$

**CCE**

$$
\text{CCE} = -\sum_{i=1}^{C} y_i \log(\hat{y}_i)
$$

**KL 散度（可用于分类/蒸馏）**

$$
D_{\text{KL}}(P \| Q) = \sum_i p_i \log \frac{p_i}{q_i}
$$


**表征学习 / 度量学习损失函数**

| 损失函数                           | 适用场景                            | 特点说明                                |
| ------------------------------ | ------------------------------- | ----------------------------------- |
| Contrastive Loss               | Siamese 网络，正负样本对                | 拉近正样本，拉远负样本，margin 控制距离             |
| Triplet Loss                   | 三元组（Anchor-Positive-Negative）结构 | 保证锚点到正样本距离小于到负样本距离 + margin         |
| InfoNCE / NT-Xent Loss         | 自监督学习（SimCLR, MoCo 等）           | 一正多负，softmax + 温度系数控制分布尖锐度，支持大批量负样本 |


**Contrastive Loss**

$$
L = y D^2 + (1 - y) \max(0, m - D)^2
$$

* $D = \| f(x_1) - f(x_2) \|_2$ ：特征空间欧氏距离
* $y \in \{0,1\}$ ：1 表示正样本，0 表示负样本
* $m$ ：margin，负样本希望距离至少大于 $m$


**Triplet Loss**

$$
L = \max(0, D(a, p) - D(a, n) + m)
$$

* $a$ ：Anchor
* $p$ ：Positive
* $n$ ：Negative
* $D(a,p) = \| f(a) - f(p) \|_2$, $D(a,n) = \| f(a) - f(n) \|_2$
* $m$ ：margin


**InfoNCE / NT-Xent Loss**

$$
L_i = - \log \frac{\exp(\text{sim}(z_i, z_i^+) / \tau)}{\sum_{j=1}^{2N} \mathbf{1}_{[j \neq i]} \exp(\text{sim}(z_i, z_j) / \tau)}
$$

* $z_i$ ：样本 $i$ 的特征向量
* $z_i^+$ ：正样本特征向量
* $z_j$ ：负样本特征向量
* $\text{sim}(\cdot, \cdot)$ ：余弦相似度
* $\tau$ ：温度系数











## 